### See if spark app is already created

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1623611811062_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-1>

In [2]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1623611811062_0002,pyspark,idle,Link,Link,✔


In [4]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- -------
beautifulsoup4             4.8.1
boto                       2.49.0
jmespath                   0.9.4
lxml                       4.4.1
mysqlclient                1.4.4
nltk                       3.4.5
nose                       1.3.4
numpy                      1.19.5
pandas                     0.23.0
pip                        21.1.2
py-dateutil                2.2
python-dateutil            2.8.1
python36-sagemaker-pyspark 1.2.6
pytz                       2019.3
PyYAML                     3.11
setuptools                 57.0.0
six                        1.12.0
soupsieve                  1.9.4
wheel                      0.36.2
windmill                   1.6

### Install Pandas if necessary

In [5]:
# install pandas
# sc.install_pypi_package("pandas==0.23")
# sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Import logs data from s3

In [6]:
dfLogs = spark.read.json("s3://udacity-de-files/raw/logs/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
dfLogs.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [8]:
dfLogs = dfLogs \
            .withColumnRenamed("firstName", "first_name") \
            .withColumnRenamed("itemInSession", "item_in_session") \
            .withColumnRenamed("lastName", "last_name") \
            .withColumnRenamed("sessionId", "session_id") \
            .withColumnRenamed("userAgent", "user_agent") \
            .withColumnRenamed("userId", "user_id") \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
import pandas as pd
pd.options.display.max_columns = 300

dfLogs.limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

        artist       auth first_name gender  item_in_session last_name  \
0     Harmonia  Logged In       Ryan      M                0     Smith   
1  The Prodigy  Logged In       Ryan      M                1     Smith   
2        Train  Logged In       Ryan      M                2     Smith   
3         None  Logged In      Wyatt      M                0     Scott   
4         None  Logged In     Austin      M                0   Rosales   

      length level                               location method      page  \
0  655.77751  free     San Jose-Sunnyvale-Santa Clara, CA    PUT  NextSong   
1  260.07465  free     San Jose-Sunnyvale-Santa Clara, CA    PUT  NextSong   
2  205.45261  free     San Jose-Sunnyvale-Santa Clara, CA    PUT  NextSong   
3        NaN  free              Eureka-Arcata-Fortuna, CA    GET      Home   
4        NaN  free  New York-Newark-Jersey City, NY-NJ-PA    GET      Home   

   registration  session_id             song  status             ts  \
0  1.541017e+12

In [10]:
dfLogs.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8056

### Create time table

In [11]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

dfTime = dfLogs \
            .filter(dfLogs.page == "NextSong") \
            .dropDuplicates(subset=["ts"]) \
            .withColumn("start_time", F.from_unixtime(F.col("ts")/1000)) \
            .withColumn("start_time", F.to_timestamp("start_time")) \
            .withColumn("hour", F.hour("start_time")) \
            .withColumn("day", F.dayofyear("start_time")) \
            .withColumn("week", F.weekofyear("start_time")) \
            .withColumn("month", F.month("start_time")) \
            .withColumn("year", F.year("start_time")) \
            .withColumn("weekday", F.dayofweek("start_time")) \
            .select("start_time", "hour", "day", "week", "month", "year", "weekday")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
dfTime.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6813

In [13]:
dfTime.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)

In [14]:
dfTime.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+---+----+-----+----+-------+
|         start_time|hour|day|week|month|year|weekday|
+-------------------+----+---+----+-----+----+-------+
|2018-11-21 10:13:42|  10|325|  47|   11|2018|      4|
|2018-11-21 11:01:05|  11|325|  47|   11|2018|      4|
|2018-11-21 18:34:01|  18|325|  47|   11|2018|      4|
|2018-11-21 20:44:26|  20|325|  47|   11|2018|      4|
|2018-11-14 03:37:52|   3|318|  46|   11|2018|      4|
+-------------------+----+---+----+-----+----+-------+
only showing top 5 rows

### Create users table

---

Falta filter usuarios unicos pelo userId

In [45]:
from pyspark.sql.functions import substring

dfUsers = dfLogs \
            .filter(dfLogs.page == "NextSong") \
            .filter(dfLogs.user_id.isNotNull()) \
            .dropDuplicates(subset=["user_id"]) \
            .withColumn("first_name_letter", substring("first_name", 1, 1)) \
            .select("user_id", "first_name", "first_name_letter", "last_name", "gender", "level") \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
dfUsers.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

96

In [47]:
dfUsers.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- first_name_letter: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

In [48]:
dfUsers.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+-----------------+---------+------+-----+
|user_id|first_name|first_name_letter|last_name|gender|level|
+-------+----------+-----------------+---------+------+-----+
|     51|      Maia|                M|    Burke|     F| free|
|      7|    Adelyn|                A|   Jordan|     F| free|
|     15|      Lily|                L|     Koch|     F| paid|
|     54|     Kaleb|                K|     Cook|     M| free|
|    101|    Jayden|                J|      Fox|     M| free|
+-------+----------+-----------------+---------+------+-----+
only showing top 5 rows

In [49]:
# Number of nulls
dfUsers.filter(dfUsers.user_id.isNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

# Import songs data from s3

In [20]:
dfSongsRaw = spark.read.json("s3://udacity-de-files/raw/songs/A/*/*/*.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
dfSongsRaw.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

71

In [22]:
dfSongsRaw.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [23]:
dfSongsRaw.limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

            artist_id  artist_latitude    artist_location  artist_longitude  \
0  ARDR4AC1187FB371A1              NaN                                  NaN   
1  AREBBGV1187FB523D2              NaN        Houston, TX               NaN   
2  ARMAC4T1187FB3FA4C         40.82624  Morris Plains, NJ         -74.47995   
3  ARPBNLO1187FB3D52F         40.71455       New York, NY         -74.00712   
4  ARDNS031187B9924F0         32.67828            Georgia         -83.22295   

                                         artist_name   duration  num_songs  \
0  Montserrat Caballé;Placido Domingo;Vicente Sar...  511.16363          1   
1       Mike Jones (Featuring CJ_ Mello & Lil' Bran)  173.66159          1   
2                          The Dillinger Escape Plan  207.77751          1   
3                                           Tiny Tim   43.36281          1   
4                                         Tim Wilson  186.48771          1   

              song_id                                   

### Create artist table

In [24]:
dfArtists = dfSongsRaw \
            .filter(dfSongsRaw.artist_id.isNotNull()) \
            .dropDuplicates(subset=["artist_id"]) \
            .withColumnRenamed("artist_name", "name") \
            .withColumnRenamed("artist_location", "location") \
            .withColumnRenamed("artist_latitude", "latitude") \
            .withColumnRenamed("artist_longitude", "longitude") \
            .select("artist_id", "name", "location", "latitude", "longitude")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
dfArtists.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

69

In [26]:
dfArtists.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)

In [27]:
dfArtists.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+--------+---------+
|         artist_id|                name|            location|latitude|longitude|
+------------------+--------------------+--------------------+--------+---------+
|AR9AWNF1187B9AB0B4|Kenny G featuring...|Seattle, Washingt...|    null|     null|
|AR0IAWL1187B9A96D0|        Danilo Perez|              Panama|  8.4177|-80.11278|
|AR0RCMP1187FB3F427|    Billie Jo Spears|        Beaumont, TX|30.08615|-94.10158|
|AREDL271187FB40F44|        Soul Mekanik|                    |    null|     null|
|ARI3BMM1187FB4255E|        Alice Stuart|          Washington| 38.8991|  -77.029|
+------------------+--------------------+--------------------+--------+---------+
only showing top 5 rows

### Create songs table

In [28]:
dfSongs = dfSongsRaw \
            .filter(dfSongsRaw.song_id.isNotNull()) \
            .dropDuplicates(subset=["song_id"]) \
            .select("song_id", "title", "artist_id", "year", "duration", "artist_name")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
dfSongs.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

71

In [30]:
dfSongs.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: long (nullable = true)
 |-- duration: double (nullable = true)
 |-- artist_name: string (nullable = true)

In [31]:
dfSongs.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+----------------+
|           song_id|               title|         artist_id|year| duration|     artist_name|
+------------------+--------------------+------------------+----+---------+----------------+
|SOGOSOV12AF72A285E|   ¿Dónde va Chichi?|ARGUVEV1187B98BA17|1997|313.12934|  Sierra Maestra|
|SOMZWCG12A8C13C480|    I Didn't Mean To|ARD7TVE1187B99BFB1|   0|218.93179|          Casual|
|SOUPIRU12A6D4FA1E1| Der Kleine Dompfaff|ARJIE2Y1187B994AB7|   0|152.92036|     Line Renaud|
|SOXVLOJ12AB0189215|     Amor De Cabaret|ARKRRTF1187B9984DA|   0|177.47546|Sonora Santanera|
|SOWTBJW12AC468AC6E|Broken-Down Merry...|ARQGYP71187FB44566|   0|151.84934|    Jimmy Wakely|
+------------------+--------------------+------------------+----+---------+----------------+
only showing top 5 rows

### Create songplays table

In [32]:
# First Join songs and artists tables first

dfSongsArtists = dfSongs.join(dfArtists, dfSongs.artist_id ==  dfArtists.artist_id, "inner")\
                        .select(dfSongs.song_id,
                                dfArtists.artist_id,
                                dfSongs.title,
                                dfSongs.duration,
                                dfArtists.name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
dfSongsArtists.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

71

In [34]:
dfSongsArtists.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- name: string (nullable = true)

In [35]:
# Now join dfSongsArtists with dfLogs
dfSongPlays = dfLogs \
                    .join(dfSongsArtists, (dfSongsArtists.name ==  dfLogs.artist) & 
                                          (dfSongsArtists.title ==  dfLogs.song) &
                                          (dfSongsArtists.duration ==  dfLogs.length), "left")

dfSongPlays = dfSongPlays \
                    .withColumn("timestamp", F.from_unixtime(F.col("ts")/1000)) \
                    .withColumn("timestamp", F.to_timestamp("timestamp")) \
                    .withColumn("year", F.year("timestamp")) \
                    .withColumn("month", F.month("timestamp")) \
                    .select("user_id",
                            "level",
                            "song_id",
                            "artist_id",
                            "session_id",
                            "location",
                            "user_agent",
                            "timestamp",
                            "year",
                            "month")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
dfSongPlays.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8056

In [37]:
dfSongPlays.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

In [38]:
dfSongPlays.limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  user_id level song_id artist_id  session_id  \
0      97  paid    None      None         537   
1      80  paid    None      None         416   
2      44  paid    None      None         157   
3      15  paid    None      None         172   
4      88  paid    None      None         888   

                                  location  \
0                 Lansing-East Lansing, MI   
1              Portland-South Portland, ME   
2                 Waterloo-Cedar Falls, IA   
3       Chicago-Naperville-Elgin, IL-IN-WI   
4  Sacramento--Roseville--Arden-Arcade, CA   

                                          user_agent           timestamp  \
0  "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5... 2018-11-13 17:35:22   
1  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4... 2018-11-09 16:03:22   
2  Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r... 2018-11-02 11:47:09   
3  "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5... 2018-11-02 12:57:34   
4  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_

# Partition by & Write to Disk

In [40]:
# https://sparkbyexamples.com/pyspark/pyspark-partitionby-example/

processed_s3_path = "s3://udacity-de-files/processed"

dfSongPlays \
        .repartition("year", "month") \
        .write \
        .partitionBy("year", "month") \
        .mode("overwrite") \
        .parquet(f"{processed_s3_path}/songplays/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
dfTime \
        .repartition("year", "month") \
        .write \
        .partitionBy("year", "month") \
        .mode("overwrite") \
        .parquet(f"{processed_s3_path}/time/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
dfSongs \
        .repartition("year", "artist_name") \
        .write \
        .partitionBy("year", "artist_name") \
        .mode("overwrite") \
        .parquet(f"{processed_s3_path}/songs/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
dfArtists \
        .repartition("name") \
        .write \
        .partitionBy("name") \
        .mode("overwrite") \
        .parquet(f"{processed_s3_path}/artists/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
# On user, first repartition by first letter of the name, then write to disk
dfUsers \
        .repartition("first_name_letter") \
        .write \
        .partitionBy("first_name_letter") \
        .mode("overwrite") \
        .parquet(f"{processed_s3_path}/users/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…